In [1]:
import numpy as np
import cv2
import os
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.auto import tqdm

/apps/software/standard/core/miniforge/24.3.0-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Grabs MRI images from 
from roboflow import Roboflow
rf = Roboflow(api_key="my key")# addd your personal api key 
project = rf.workspace("alzheimer-h49wu").project("alzheimer-detection-ra7oh")
version = project.version(1)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to alzheimer-detection-1 in folder:: 100%|██████████| 9778/9778 [00:16<00:00, 583.62it/s]


In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the base directory 
base_dir = '/home/rkk6nx/Project3/CS3/alzheimer-detection-1'  # Update this path to where your data is located

# Define the directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# The categories in your dataset
labels = ['Mild Impairment', 'No Impairment', 'Moderate Impairment', 'Very Mild Impairment']

# Create a new directory to combine train and test data
combined_dir = os.path.join(base_dir, 'combined')
os.makedirs(combined_dir, exist_ok=True)

# Combine images from train and test for each impairment type
for label in labels:
    # Paths for each category (train and test)
    label_train_dir = os.path.join(train_dir, label)
    label_test_dir = os.path.join(test_dir, label)

    # Create the combined directory for this label
    label_combined_dir = os.path.join(combined_dir, label)
    os.makedirs(label_combined_dir, exist_ok=True)

    # Get a list of all images in both the train and test directories
    train_images = [os.path.join(label_train_dir, f) for f in os.listdir(label_train_dir) if f.endswith('.jpg')]
    test_images = [os.path.join(label_test_dir, f) for f in os.listdir(label_test_dir) if f.endswith('.jpg')]

    # Combine images
    all_images = train_images + test_images

    # Copy all images to the combined directory for this label
    for img in all_images:
        shutil.copy(img, label_combined_dir)

print("Images have been successfully combined into the 'combined' folder!")

# Now resplit the combined data into a new train/test split
new_train_dir = os.path.join(base_dir, 'new_train')
new_test_dir = os.path.join(base_dir, 'new_test')
os.makedirs(new_train_dir, exist_ok=True)
os.makedirs(new_test_dir, exist_ok=True)

# Split images into new train/test set for each label
for label in labels:
    label_combined_dir = os.path.join(combined_dir, label)

    # Get all image paths for the label
    all_images = [os.path.join(label_combined_dir, f) for f in os.listdir(label_combined_dir) if f.endswith('.jpg')]

    # Split into train/test (80% train, 20% test) using sklearn's train_test_split
    train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)

    # Create directories for the new train and test splits
    label_new_train_dir = os.path.join(new_train_dir, label)
    label_new_test_dir = os.path.join(new_test_dir, label)
    os.makedirs(label_new_train_dir, exist_ok=True)
    os.makedirs(label_new_test_dir, exist_ok=True)

    # Move images to new train and test directories
    for img in train_images:
        shutil.copy(img, label_new_train_dir)
    
    for img in test_images:
        shutil.copy(img, label_new_test_dir)

print("Images have been successfully resplit into new train/test sets!")


Images have been successfully combined into the 'combined' folder!
Images have been successfully resplit into new train/test sets!
